In [1]:
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, Trainer, RobertaTokenizer, LongformerForSequenceClassification
import torch
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from evaluate import evaluator
import torch.cuda
import pandas as pd

2024-04-11 15:09:14.467552: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 15:09:14.467595: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 15:09:14.469919: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 15:09:14.712384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 15:09:16.022945: W tensorflow/compiler/

In [2]:
echr = load_dataset("ecthr_cases",  "violation-prediction")

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [4]:
def encode(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     padding=True)

In [5]:
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": "\n".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map(encode, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {'labels' :list(1 if examples['labels'][i] else 0 for i in range(len(examples['labels'])))}, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
train_df, test_df, val_df = pd.DataFrame(train_dataset), pd.DataFrame(test_dataset), pd.DataFrame(val_dataset)

In [7]:
train_df2 = Dataset.from_pandas(pd.concat([train_df[train_df['labels'] == 1].sample(762, random_state=42), train_df[train_df['labels'] == 0]]).sort_index())

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2, id2label=id2label, label2id=label2id)


/home/leontioso/anaconda3/envs/msc_thesis/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="../models/longformer_ecthr_model",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_df2,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [14]:

trainer.train()

Initializing global attention on CLS token...


Epoch,Training Loss,Validation Loss,F1
1,No log,0.591395,0.796484
2,No log,0.647651,0.830709


TrainOutput(global_step=382, training_loss=0.6243124357692859, metrics={'train_runtime': 6195.337, 'train_samples_per_second': 0.492, 'train_steps_per_second': 0.062, 'total_flos': 1001041856151552.0, 'train_loss': 0.6243124357692859, 'epoch': 2.0})

In [15]:
task_evaluator = evaluator("text-classification")
results_dict = {}
for metric in ["accuracy", "precision", "recall", "f1"]:
    results = task_evaluator.compute(
        model_or_pipeline="../models/longformer_ecthr_model/checkpoint-382",
        data=test_dataset,
        metric=metric,
        tokenizer=tokenizer,
        strategy="simple",
        random_state=0,
        input_column='text',
        label_column='labels',
        label_mapping={"NON_VIOLATED": 0.0, "VIOLATED": 1.0},
    )
    metric_name, value = list(results.items())[0]
    results_dict[metric_name] = value

Input ids are automatically padded from 441 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 264 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 172 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 417 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 428 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 153 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 439 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 284 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 226 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 260 to 512 to be a multiple of `config.att

In [16]:
results_dict

{'accuracy': 0.753,
 'precision': 0.9439655172413793,
 'recall': 0.7595375722543353,
 'f1': 0.8417680973734785}

In [56]:
len(val_dataset[0]["input_ids"])

18291